In [2]:
import os 

# Move to the survey results directory 
data_dir = '../../data'
os.chdir(data_dir)

In [13]:
####################
# Remove responses with a yes in the remove column 
##################

import csv

def filter_responses(input_file, accepted_file, rejected_file):
    encodings = ['utf-8', 'ISO-8859-1', 'windows-1252']
    
    for encoding in encodings:
        try:
            with open(input_file, 'r', newline='', encoding=encoding) as infile, \
                 open(accepted_file, 'w', newline='', encoding='utf-8') as accepted_outfile, \
                 open(rejected_file, 'w', newline='', encoding='utf-8') as rejected_outfile:
                
                reader = csv.reader(infile)
                accepted_writer = csv.writer(accepted_outfile)
                rejected_writer = csv.writer(rejected_outfile)
                
                # Write header row if it exists
                header = next(reader, None)
                if header:
                    accepted_writer.writerow(header)
                    rejected_writer.writerow(header)
                
                # Filter and write rows
                for row in reader:
                    if row:
                        if not row[0].lower().startswith('yes'):
                            accepted_writer.writerow(row)
                        else:
                            rejected_writer.writerow(row)
                
                print(f"Successfully processed the file with {encoding} encoding.")
                return  # Exit the function if successful
        except UnicodeDecodeError:
            print(f"Failed to read with {encoding} encoding. Trying next...")
    
    print("Failed to read the file with any of the specified encodings.")
    raise

# Usage
input_file = 'AI_Study_Prolific_Reviewed.csv'  # Use the output file from the previous merge script
accepted_file = 'AI_Study_Prolific_Reviewed_Accepted.csv'
rejected_file = 'rejected_responses.csv'

filter_responses(input_file, accepted_file, rejected_file)
print(f"Filtered CSVs created: {accepted_file} and {rejected_file}")

Successfully processed the file with utf-8 encoding.
Filtered CSVs created: AI_Study_Prolific_Reviewed_Accepted.csv and rejected_responses.csv


In [1]:
import csv

def merge_csv_files(file1_path, file2_path, output_path, skip_rows=0):
    # Read the first file
    with open(file1_path, 'r', newline='', encoding='utf-8') as file1:
        reader1 = csv.reader(file1)
        data1 = list(reader1)

    # Read the second file, skipping the specified number of rows
    with open(file2_path, 'r', newline='', encoding='utf-8') as file2:
        reader2 = csv.reader(file2)
        # Skip the specified number of rows
        for _ in range(skip_rows):
            next(reader2, None)
        data2 = list(reader2)

    # Merge the data
    merged_data = data1 + data2

    # Write the merged data to the output file
    with open(output_path, 'w', newline='', encoding='utf-8') as output_file:
        writer = csv.writer(output_file)
        writer.writerows(merged_data)

    print(f"Merged file created: {output_path}")

# Usage example
file1_path = 'AI_Study_Accepted_With_replacement_codes.csv'
file2_path = 'AI_Study_Qualtric_Accepted_With_replacement_codes.csv'
output_path = 'AI_Study_Accepted_With_replacement_codes.csv'
skip_rows = 1  # Change this to the number of rows you want to skip in the second file

merge_csv_files(file1_path, file2_path, output_path, skip_rows)

Merged file created: AI_Study_Accepted_With_replacement_codes.csv


In [18]:
import csv
import json

def update_json_with_new_rows(json_file, new_csv_file, updated_json_file):
    # Read the existing JSON data
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Create a mapping of text entries to their new row numbers from the new CSV
    new_row_map = {}
    with open(new_csv_file, 'r', newline='', encoding='utf-8') as infile:
        reader = csv.reader(infile)
        header = next(reader)  # Skip header row
        for row_num, row in enumerate(reader, start=3):  # Start from 3 to match your original indexing
            # Assuming the text is in the first column (index 0)
            text_value = row[0].strip()  # Strip whitespace for accurate matching
            new_row_map[text_value] = row_num  # Map text to its new row number

    # Update the JSON data with new row numbers
    for entry in data.keys():
        for item in data[entry]:
            # Strip whitespace from the text for accurate matching
            item_text = item['text'].strip()
            if item_text in new_row_map:
                item['row_number'] = new_row_map[item_text]  # Update the row number

    # Write the updated JSON data to a new file
    with open(updated_json_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

    print(f"Updated JSON created: {updated_json_file}")

# Usage
json_file_path = 'Prolific_demographics_other_with_row.json'  # Existing JSON file
new_csv_file_path = 'AI_Study_Accepted_With_Replacement_Codes.csv'  # New CSV file with updated entries
updated_json_file_path = 'Updated_Prolific_demographics.json'  # Output updated JSON file

update_json_with_new_rows(json_file_path, new_csv_file_path, updated_json_file_path)

Updated JSON created: Updated_Prolific_demographics.json


In [4]:
import json

def remove_questions_by_index(input_file, output_file, indices_to_remove):
    """
    Remove specified questions from an annotator book based on their index numbers.
    
    Args:
        input_file (str): Path to the input JSON annotator book
        output_file (str): Path where the filtered JSON will be saved
        indices_to_remove (list): List of index numbers to remove (e.g., ['4.7', '4.8', '10.1'])
                                Can be provided with or without the section letter
    """
    # Read the annotator book
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Create new dict for filtered data
    filtered_data = {}
    removed_questions = []
    
    # Process indices to handle both formats (e.g., '4.7' and 'B.4.7')
    formatted_indices = []
    for idx in indices_to_remove:
        # Remove section letter if present and any leading/trailing whitespace
        cleaned_idx = idx.strip()
        if '.' in cleaned_idx and cleaned_idx[0].isalpha():
            cleaned_idx = cleaned_idx[cleaned_idx.index('.')+1:]
        formatted_indices.append(cleaned_idx)
    
    # Filter the questions
    for question, responses in data.items():
        # Extract the index from the question
        # Look for the pattern B.X.Y: or X.Y:
        question_parts = question.split(':')[0].strip()  # Get everything before the colon
        if question_parts[0].isalpha():
            question_index = question_parts[question_parts.index('.')+1:]  # Remove section letter
        else:
            question_index = question_parts
            
        # If the question index is not in our remove list, keep it
        if question_index.strip() not in formatted_indices:
            filtered_data[question] = responses
        else:
            removed_questions.append(question)
    
    # Write the filtered data to the output file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(filtered_data, f, indent=2, ensure_ascii=False)
    
    # Print statistics
    print(f"\nQuestion removal completed successfully!")
    print(f"Output saved to: {output_file}")
    print(f"\nStatistics:")
    print(f"Total questions removed: {len(removed_questions)}")
    print("\nRemoved questions:")
    for q in removed_questions:
        print(f"- {q.split(':')[0].strip()}")

# Example usage:

indices_to_remove_B = ['B.9.1', 'B.9.2', 'B.9.3','B.9.4','B.10.1', 'B.10.2', 'B.10.3','B.11.1', 'B.11.2', 'B.11.3','B.11.4','B.12.1', 'B.12.2', 'B.12.3','B.12.4','B.13.1', 'B.13.2', 'B.13.3','B.13.4','B.14.1', 'B.14.2', 'B.14.3','B.14.4','B.16.1', 'B.16.2', 'B.16.5','B.16.4','B.17.1', 'B.17.2', 'B.17.3','B.18','C.1','C.2']

'''indices_to_remove_A = [
    'A.4','A.2.2Â','A.2.1Â _2','A.2.1Â _2',
    'A.2.1Â _2','A.2.1Â _1','A.2.1Â _3','A.2.1Â _4','A.2.1Â _5','A.2.1Â _6','A.2.1Â _7','A.2.1Â _8','A.2.1Â _9',
    'A.1.5','A.1.4','A.1.2','A.1.3',
    'A.4.1','A.4.2','A.4.3'
]'''"AI Study_Qualtric-203-all.csv"


indices_to_remove_B_non_risk_Qs = ['B.1.1','B.2.5_16_TEXT','B.2.6','B.11.1_8_TEXT','B.11.4_8_TEXT','B.11.5','B.11.6','B.12.5','B.12.6','B.13.5','B.13.6','B.14.5','B.14.6','B.9.5','B.9.6','B.10.4','B.16.2_7_TEXT','B.16.3','B.16.4_9_TEXT','B.16.5_5_TEXT','B.16.6','B.17.4','B.17.5','C.3_1_TEXT']

indices_to_remove_A_non_risk_Qs = ['A.1.1','A.1.2','A.1.4_14_TEXT','A.1.5_6_TEXT','A.1.6','A.4.3_14_TEXT']

indices_to_remove = indices_to_remove_B_non_risk_Qs

remove_questions_by_index(
    'Annotator_A_all.json',
    'Annotator_A_Wilder_Risk_Mitigation.json',
    indices_to_remove
)



Question removal completed successfully!
Output saved to: Annotator_A_Wilder_Risk_Mitigation.json

Statistics:
Total questions removed: 1

Removed questions:
- A.1.1


In [ ]:
import json
import re

def format_annotator_book(input_file, output_file):
    """
    Reformats an annotator book JSON file to ensure only themes and codes arrays
    appear on single lines, without adding extra commas.
    
    Args:
        input_file (str): Path to the input JSON annotator book
        output_file (str): Path where the reformatted JSON will be saved
    """
    # First read the file and parse JSON
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Write initial JSON with standard formatting
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    
    # Read the formatted content
    with open(output_file, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Function to clean and format array content
    def format_array(match):
        array_content = match.group(1)
        # Split into lines and clean up each line
        elements = []
        for line in array_content.split('\n'):
            line = line.strip()
            if line and not line.startswith('[') and not line.endswith(']'):
                # Remove any trailing commas
                if line.endswith(','):
                    line = line[:-1]
                elements.append(line)
        
        # Join elements with proper comma separation
        return '[' + ', '.join(elements) + ']'
    
    # Replace theme arrays
    content = re.sub(r'"theme": \[(.*?)\]',
                    lambda m: '"theme": ' + format_array(m),
                    content,
                    flags=re.DOTALL)
    
    # Replace codes arrays
    content = re.sub(r'"codes": \[(.*?)\]',
                    lambda m: '"codes": ' + format_array(m),
                    content,
                    flags=re.DOTALL)
    
    # Write the final formatted content
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(content)

    # Verify the JSON is valid
    try:
        with open(output_file, 'r', encoding='utf-8') as f:
            json.load(f)
        print("\nFormatting completed successfully!")
        print("Output saved to: {}".format(output_file))
    except json.JSONDecodeError as e:
        print("Error details:", str(e))

import os 
for file in os.listdir('../../data/annotations/json'):
    # Example usage:
    format_annotator_book(
        f'../../data/annotations/json/{file}',
        f'../../data/annotations/json/{file}'
    )


Formatting completed successfully!
Output saved to: ../../data/annotations/json/AI_study_demographics_other.json

Formatting completed successfully!
Output saved to: ../../data/annotations/json/A_definitions.json

Formatting completed successfully!
Output saved to: ../../data/annotations/json/A_no_adoption.json

Formatting completed successfully!
Output saved to: ../../data/annotations/json/A_risk_mitigation.json

Formatting completed successfully!
Output saved to: ../../data/annotations/json/B_definitions.json

Formatting completed successfully!
Output saved to: ../../data/annotations/json/B_risk_mitigation.json

Formatting completed successfully!
Output saved to: ../../data/annotations/json/B_role_am.json


PermissionError: [Errno 13] Permission denied: '../../data/annotations/json/demographics_other'

In [5]:
import csv


def reorder_csv(input_file, output_file):
    with open(input_file, 'r', newline='', encoding='utf-8') as infile:
        reader = csv.reader(infile)
        
        # Read the first 3 rows (including header)
        first_three_rows = [next(reader) for _ in range(3)]
        
        # Separate remaining rows with text in the first column from those without
        rows_with_text = []
        rows_without_text = []
        
        for row in reader:
            if row and row[0].strip():  # Check if the first column has any non-whitespace text
                rows_with_text.append(row)
            else:
                rows_without_text.append(row)
    
    # Write the reordered data to the output file
    with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        writer.writerows(first_three_rows)  # Write the first 3 rows as they were
        writer.writerows(rows_with_text)
        writer.writerows(rows_without_text)

    print(f"Reordered CSV created: {output_file}")

# Usage
input_file = 'AI_Study_Prolific_Reviewed.csv'  # Replace with your input CSV file path
output_file = 'AI_Study_Prolific_Reordered.csv'  # Output CSV file path

reorder_csv(input_file, output_file)


Formatting completed successfully!
Output saved to: Annotator_B_C.json
Reordered CSV created: AI_Study_Prolific_Reordered.csv


In [46]:
import json
from collections import Counter
from typing import Dict, List, Tuple

def analyze_json(data: Dict) -> Tuple[Dict[str, int], Dict[str, int], int]:
    themes_counter = Counter()
    codes_counter = Counter()
    total_responses = 0

    for question, responses in data.items():
        for response in responses:
            total_responses += 1
            # Normalize themes to lowercase before counting
            themes_counter.update([theme.lower() for theme in response['theme']])  
            codes_counter.update(response['codes'])

    return themes_counter, codes_counter, total_responses

def organize_results(themes_counter: Counter, codes_counter: Counter, data: Dict) -> Dict:
    output = {}
    for theme_lower in themes_counter.keys(): 
        theme = theme_lower.capitalize()  
        code_counts = Counter()
        for question, responses in data.items():
            for response in responses:
                if theme_lower in [t.lower() for t in response['theme']]:  # Compare lowercase
                    code_counts.update(response['codes'])
        output[theme] = [{"code": code, "count": count} for code, count in code_counts.items()]
    return output

# Main execution
with open('Annotator_B_Ersi-Wilder_Risk_Mitigation.json', 'r', encoding='latin-1') as file:
    data = json.load(file)

themes_counter, codes_counter, total_responses = analyze_json(data)
organized_results = organize_results(themes_counter, codes_counter, data)

# Save the results to a JSON file
with open('themes_and_codes.json', 'w') as outfile:
    json.dump(organized_results, outfile, indent=4)

print("Analysis complete. Results saved to themes_and_codes.json")

Analysis complete. Results saved to themes_and_codes.json


In [6]:
import json
import pandas as pd
from collections import defaultdict

# Load and parse JSON data
with open('annotator_B_C.json', 'r',encoding='latin-1') as file:
    try:
        data = json.load(file)
    except json.JSONDecodeError:
        print("Error decoding JSON. Please check the file format.")
        raise

# Dictionary to store themes and examples
themes_dict = defaultdict(list)

# Process each key in the JSON data
for key in data:
    entries = data[key]  # Extract list of entries

    for entry in entries:
        text = entry.get("text", "")
        themes = entry.get("theme", [])
        codes = entry.get("codes", [])
        
        # Associate text examples with each theme, storing text and code count
        for theme in themes:
            themes_dict[theme].append({
                "text": text,
                "codes": codes
            })

# Prepare data for CSV output
csv_data = []

for theme, examples in themes_dict.items():
    # Sort examples for each theme by the number of codes in descending order
    sorted_examples = sorted(examples, key=lambda x: len(x["codes"]), reverse=True)
    
    # Get the example with the most codes
    if sorted_examples:
        top_example = sorted_examples[0]
        codes_str = ", ".join(top_example["codes"])  # Combine all codes for column 1
        example_text = top_example["text"]  # Column 3: Text of the top entry
        theme_desc = ""  # Placeholder for description
        
        # Add to CSV data with all required columns
        csv_data.append([codes_str, theme, example_text, theme_desc])

# Convert to DataFrame and save as CSV
df = pd.DataFrame(csv_data, columns=["Codes", "Theme", "Example Text", "Description"])
df.to_csv("themes_with_top_example.csv", index=False)

print("CSV file created: themes_with_top_example.csv")


CSV file created: themes_with_top_example.csv


In [3]:
import json
import re
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch

def remove_non_ascii(text):
    # Remove non-ASCII characters from the text
    return re.sub(r'[^\x00-\x7F]+', '', text)

def create_pdf(data_list, pdf_filename):
    # Set up the PDF document
    doc = SimpleDocTemplate(pdf_filename, pagesize=letter)
    elements = []
    styles = getSampleStyleSheet()
    heading_style = styles['Heading2']
    normal_style = styles['BodyText']
    
    # Increase spacing in heading and normal styles
    heading_style.spaceAfter = 14
    normal_style.spaceAfter = 8
    normal_style.leading = 14  # Increase line height for readability

    # Merge the data from both files
    merged_data = {}
    for data in data_list:
        for question, answers in data.items():
            if question not in merged_data:
                merged_data[question] = answers
            else:
                merged_data[question].extend(answers)

    for question, answers in merged_data.items():
        # Remove non-ASCII characters from the question text
        clean_question = remove_non_ascii(question)
        # Add the question as a heading
        elements.append(Paragraph(f"<b>{clean_question}</b>", heading_style))
        elements.append(Spacer(1, 10))

        # Collect answers in table data
        table_data = []
        for item in answers:
            # Remove non-ASCII characters from the answer text
            text = remove_non_ascii(item.get('text', '').strip())
            if text:
                p = Paragraph(text, normal_style)
                table_data.append([p])

        # Create table if there are answers
        if table_data:
            # Create the table with improved padding and alternate row colors
            table = Table(table_data, colWidths=[6.5*inch])
            row_count = len(table_data)
            bg_colors = [colors.whitesmoke, colors.lightgrey]
            ts = TableStyle(
                [('BACKGROUND', (0, i), (-1, i), bg_colors[i % 2]) for i in range(row_count)] +
                [('VALIGN', (0, 0), (-1, -1), 'TOP'),
                 ('FONT', (0, 0), (-1, -1), 'Helvetica', 10),
                 ('LEFTPADDING', (0, 0), (-1, -1), 8),
                 ('RIGHTPADDING', (0, 0), (-1, -1), 8),
                 ('TOPPADDING', (0, 0), (-1, -1), 6),
                 ('BOTTOMPADDING', (0, 0), (-1, -1), 6)]
            )
            table.setStyle(ts)
            elements.append(table)
            elements.append(Spacer(1, 20))

    # Build the PDF
    doc.build(elements)

# Read JSON data from two files
file_names = ['All_As.json']
data_list = []

for file_name in file_names:
    with open(file_name, 'r', encoding='utf-8') as f:
        data = json.load(f)
        data_list.append(data)

create_pdf(data_list, 'Annotations/all_survey_results.pdf')


In [2]:
########################################
# Split CSV with A track and B track
########################################

import pandas as pd

def split_csv(input_file, track_a_file, track_b_file):
    """
    Splits the input CSV into two separate CSV files (track_a and track_b) based on the presence
    of text in columns 42 to 60. Both output files retain the first two rows of the input file.

    Parameters:
    - input_file (str): Path to the input CSV file.
    - track_a_file (str): Path for the first output CSV file (track_a).
    - track_b_file (str): Path for the second output CSV file (track_b).
    """
    # Read the CSV file without assuming a header and treat all data as strings
    df = pd.read_csv(input_file, dtype=str, keep_default_na=False, header=None)

    # Extract the first two rows to serve as headers for both output files
    header = df.iloc[:2]

    # Extract the remaining data
    data = df.iloc[2:]

    # Define the column indices for columns 42 to 60 (0-based indexing)
    col_start = 55  # Column 42
    col_end = 65   # Column 60 (exclusive in pandas iloc)

    # Determine the actual end index based on the dataframe's columns
    max_cols = data.shape[1]
    col_end = min(col_end, max_cols)

    if col_start >= max_cols:
        # If there are fewer than 42 columns, no rows will match the condition
        mask = pd.Series([False] * len(data))
    else:
        # Create a boolean mask where True indicates that at least one of the columns 42-60 has text
        mask = data.iloc[:, col_start:col_end].apply(lambda row: row.str.strip().astype(bool), axis=1).any(axis=1)

    # Split the data based on the mask
    track_a_data = data[mask]
    track_b_data = data[~mask]

    # Concatenate the headers with the respective data
    track_a = pd.concat([header, track_a_data], ignore_index=True)
    track_b = pd.concat([header, track_b_data], ignore_index=True)

    # Write the split data to the respective CSV files without headers and index
    track_a.to_csv(track_a_file, index=False, header=False)
    track_b.to_csv(track_b_file, index=False, header=False)

    print(f"Split completed successfully!")
    print(f"Rows with text in columns 42-60: {len(track_a_data)} written to {track_a_file}")
    print(f"Remaining rows: {len(track_b_data)} written to {track_b_file}")


if __name__ == "__main__":
    # Define your file paths
    input_csv = 'AI_Study_Accepted_With_Replacement_Codes.csv'       # Replace with your input file path
    output_track_a = 'track_a.csv'  # Desired output path for track A
    output_track_b = 'track_b.csv'  # Desired output path for track B

    # Call the function to perform the split
    split_csv(input_csv, output_track_a, output_track_b)


Split completed successfully!
Rows with text in columns 42-60: 107 written to track_a.csv
Remaining rows: 307 written to track_b.csv
